In [ ]:
import requests
from requests_ntlm import HttpNtlmAuth
import json
import csv
import netrc
import pandas as pd
import numpy as np

In [ ]:
# get user / password
host = 'torvmpiis8'
parking_url = 'http://torvmpiis8.na.sas.com/ParkingApp/BookingCalendar/GetBooking'
# Read from the .netrc file in your home directory
secrets = netrc.netrc()
dusername, account, password = secrets.authenticators(host)
#print ("\"" + dusername + "\" \"" + password + "\"")

In [ ]:
try:
        bookings = requests.get(parking_url,auth=HttpNtlmAuth(dusername,password))
        bookings.raise_for_status()
except requests.exceptions.HTTPError as err:
        print ("failed to make server connection[parking URL]: " + str(err))
        sys.exit(1)

In [ ]:
# parse to json
data = json.loads(bookings.text)
spots = data["BList"]

### Run this one if we want to update the file

In [ ]:
## write to CSV
with open('spots.csv', 'w') as csvfile:
        fieldnames = ['pspot','name','date','bookOrPost','period','spot']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, lineterminator='\n')
        writer.writeheader()
        for spot in spots:
                spot['pspot'] = spot['spot'].split(' ')[0]
                spot['name'] = spot['spot'].split(' ')[1]
                spot['period'] = (spot['spot'].split('(')[1]).split(' ')[0]
                writer.writerow(spot)

In [ ]:
## Create a lookup table of SpotIds to Names
spot_range = pd.DataFrame(pd.Series(np.arange(1,64)), columns=['SpotId'])
spot_names = pd.read_csv("spot_names.csv", " ")
names['SpotId'] =  names['SpotId'].astype(int)
spot_name_lookup = pd.merge(spot_range,spot_names, how='left', on='SpotId')
spot_name_lookup